In [ ]:
import os
import tiktoken
from dotenv import load_dotenv
from tavily import TavilyClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from llama_index.core.agent import ReActAgent
from llama_index.core.callbacks import (
    CallbackManager,
    TokenCountingHandler,
    LlamaDebugHandler,
)
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.tools import FunctionTool
from llama_index.core import Settings
from llama_index.llms.azure_openai import AzureOpenAI
from utils.search import search_web_tool
from utils.retrieval import retrieval_tool

# Load environment variables
load_dotenv()

# Set up OpenAI and Tavily clients

In [2]:
gpt4o_client = AzureOpenAI(
    model=os.getenv("AZURE_OPENAI_MODEL"),
    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_GPOT4O_ENDPOINT"),
    temperature=os.getenv("AZURE_OPENAI_TEMPERATURE"),
)

In [3]:
tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

# Set up Search client

In [4]:
search_client = SearchClient(
    endpoint=os.getenv("AI_SEARCH_ENDPOINT"),
    index_name=os.getenv("INDEX_NAME"),
    credential=AzureKeyCredential(os.getenv("AI_SEARCH_API_KEY")),
)

# Set up default model for Llama index

In [5]:
Settings.llm = gpt4o_client

# Initialize memory buffer and callback manager

In [6]:
token_counter = TokenCountingHandler(tokenizer=tiktoken.encoding_for_model(os.getenv("AZURE_OPENAI_MODEL")).encode, verbose=True)
llama_debug = LlamaDebugHandler(print_trace_on_end=True)

callback_manager = CallbackManager([token_counter, llama_debug])

chat_memory = ChatMemoryBuffer(token_limit=32000, )

# Set up tools

In [7]:
search_tool = FunctionTool.from_defaults(
    name="search_tool",
    description="Search the web for additional information when required.",
    fn= lambda query: search_web_tool(query=query, tavily=tavily)
)


In [8]:
retriever_tool = FunctionTool.from_defaults(
    name="retrieval_tool",
    description="Search the Azure Cognitive Search for additional information when the topic is Tesla's risk or revenue forecast",
    fn= lambda query: retrieval_tool(query=query, search_client=search_client, gpt_client=gpt4o_client),
)

# Set up ReAct Agent

In [9]:
agent = ReActAgent.from_tools(
    llm=gpt4o_client,
    tools=[search_tool, retriever_tool],
    #memory=chat_memory,
    callback_manager=callback_manager,
    verbose=True,
    max_iterations=10
)

In [12]:
# Reset agent if new session is required
agent.reset()

# Interact with the agent

In [ ]:
response = agent.chat("what are the main risks Tesla is facing?")
print(response)